In [23]:
import os
# Change the working directory to strategy_lab
os.chdir("/Users/sugang/Desktop/projects/2024coin/strategy_lab")


import pandas as pd
import numpy as np
from handle_candle import resample_df
from datetime import datetime
from performance_utils import get_performance


In [24]:
def backtest(df, window_length=14):
    # Calculate the price change
    df["price_change"] = df["close"].diff()

    # Calculate gains and losses
    df["gain"] = np.where(df["price_change"] > 0, df["price_change"], 0)
    df["loss"] = np.where(df["price_change"] < 0, -df["price_change"], 0)

    # Calculate average gain and average loss over a 14-period window
    df["avg_gain"] = df["gain"].rolling(window=window_length).mean()
    df["avg_loss"] = df["loss"].rolling(window=window_length).mean()

    # Calculate RS and RSI
    df["rs"] = df["avg_gain"] / df["avg_loss"]
    df["rsi"] = 100 - (100 / (1 + df["rs"]))

    # Set default signal to 0, no position
    df["signal"] = 0
    df.loc[(df["rsi"] >= 30) & (df["rsi"].shift(1).fillna(0) < 30), "signal"] = 1  # Buy signal
    df.loc[(df["rsi"] <= 70) & (df["rsi"].shift(1).fillna(0) > 70), "signal"] = -1  # Sell signal

    # Initialize the position column with default value 0
    df["position"] = 0
    
    # Variable to track the current position status
    current_position = 0
    
    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position = 0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1:  # Exit position
            df.loc[i, "position"] = current_position
            current_position = 0
            continue
        df.loc[i, "position"] = current_position
    
    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]
    
    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)
    
    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)
    
    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    return {"strategy": f"rsi_{window_length}", "stoploss": None, **performance}

def backtest_stoploss(df, window_length=14, stop_loss_pct=0.05):
    # Calculate the price change
    df["price_change"] = df["close"].diff()

    # Calculate gains and losses
    df["gain"] = np.where(df["price_change"] > 0, df["price_change"], 0)
    df["loss"] = np.where(df["price_change"] < 0, -df["price_change"], 0)

    # Calculate average gain and average loss over a 14-period window
    df["avg_gain"] = df["gain"].rolling(window=window_length).mean()
    df["avg_loss"] = df["loss"].rolling(window=window_length).mean()

    # Calculate RS and RSI
    df["rs"] = df["avg_gain"] / df["avg_loss"]
    df["rsi"] = 100 - (100 / (1 + df["rs"]))

    # Set default signal to 0, no position
    df["signal"] = 0
    df.loc[(df["rsi"] >= 30) & (df["rsi"].shift(1).fillna(0) < 30), "signal"] = 1  # Buy signal
    df.loc[(df["rsi"] <= 70) & (df["rsi"].shift(1).fillna(0) > 70), "signal"] = -1  # Sell signal

    # Initialize the position column with default value 0
    df["position"] = 0
    df["highest_price"] = np.nan
    
    # Variable to track the current position status
    current_position = 0
    
    # Iterate over the rows and set the position
    for i in range(1, len(df)):
        if df.loc[i-1, "signal"] == 1:  # Enter long position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "close"], df.loc[i, "close"])
            if df.loc[i, "signal"] == -1:
                df.loc[i, "position"] = 1
                current_position = 0
                continue
                
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position = 0
                continue
            current_position = 1
        elif df.loc[i, "signal"] == -1 and current_position == 1:  # Exit position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            df.loc[i, "position"] = current_position
            current_position = 0
            continue
        
        elif current_position == 1:  # Check current_position instead of df position
            df.loc[i, "highest_price"] = max(df.loc[i-1, "highest_price"], df.loc[i, "close"])
            if df.loc[i, "close"] <= df.loc[i, "highest_price"] * (1 - stop_loss_pct):
                df.loc[i, "position"] = 1
                df.loc[i, "signal"] = -1
                current_position = 0
                continue
        
        if current_position == 0 and df.loc[i, "rsi"] > 30 and df.loc[i-1, "rsi"] <= 30:
            df.loc[i, "signal"] = 1
        df.loc[i, "position"] = current_position
    
    # Calculate the strategy returns (only when in a long position)
    df["strategy_returns"] = df["position"] * df["close"].pct_change()
    df["strategy_returns2"] = df["strategy_returns"]
    
    # Adjust for trading fees (buy with 0.2% fee, sell with 0.2% fee)
    df["buy_price"] = df["close"].shift(1) * np.where(df["signal"].shift(1) == 1, 1.002, 1)
    df["sell_price"] = df["close"] * np.where(df["signal"] == -1, 0.998, 1)
    
    # Calculate strategy returns with fees
    df["strategy_returns2"] = np.where(df["position"] == 1, df["sell_price"] / df["buy_price"] - 1, 0)
    
    # Calculate the cumulative returns
    df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    df.to_csv(f"6_relative_strength_index/temp_stoploss_{window_length}_{stop_loss_pct}.csv")
    return {"strategy": f"rsi_{window_length}", "stoploss": stop_loss_pct, **performance}

def backtest_benchmark(df):
    df = df.copy()  
    df["strategy_returns2"] = df["close"].pct_change()
    df["cumulative_returns2"] = (1 + df["strategy_returns2"]).cumprod()
    performance = get_performance(df)
    # df.to_csv(f"6_relative_strength_index/temp_benchmark.csv")
    return {"strategy": "benchmark", "stoploss": None, **performance}


In [25]:
# Parameters
window_lengths = [5, 10, 14, 20, 50]
stop_loss_pcts = [0.03, 0.05, 0.1, 0.2]
hours = range(0, 24, 1)

# Store results in a list
results = []
# Read the data
df = pd.read_csv("temp.csv", index_col=0)
# execution_time = datetime.strptime("00:00", "%H:%M")
# df = resample_df(df, execution_time=execution_time)
df_copy = df.copy()
df_copy2 = df.copy()
execution_time = datetime.strptime("19:00", "%H:%M")
df_copy = resample_df(df_copy, execution_time=execution_time)
df_copy2 = resample_df(df_copy2, execution_time=execution_time)
print(backtest(df_copy, window_length=14))
print(backtest_stoploss(df_copy2, window_length=14, stop_loss_pct=0.05))

# # Run backtests and collect results
# results.append(backtest_benchmark(df))

# df = pd.read_csv("temp.csv", index_col=0)
# for hour in hours:
#     df_copy = df.copy()
#     execution_time = datetime.strptime(f"{hour:02d}:00", "%H:%M")
#     df_copy = resample_df(df_copy, execution_time=execution_time)
#     for window_length in window_lengths:
#         results.append({
#             "execution_time": execution_time.strftime("%H:%M"),
#             **backtest(df_copy, window_length)
#         })
#         for stop_loss_pct in stop_loss_pcts:
#             results.append({
#                 "execution_time": execution_time.strftime("%H:%M"),
#                 **backtest_stoploss(df_copy, window_length, stop_loss_pct)
#             })

# # Save results
# results_df = pd.DataFrame(results)
# results_df.to_csv("6_relative_strength_index/results.csv", index=False)
# print("Results saved to results.csv")


{'strategy': 'rsi_14', 'stoploss': None, 'total_return': 0.37554647292521937, 'cagr': 0.045886993056728054, 'mdd': 0.7378126747275834, 'win_rate': 0.6206896551724138, 'buy_time': 29, 'win_time': 18, 'gain_loss_ratio': 0.99, 'holding_time_ratio': 0.5057825751734772, 'investing_period': 2594, 'sharpe_ratio': 0.26373349856403056}
{'strategy': 'rsi_14', 'stoploss': 0.05, 'total_return': 2.891530579713319, 'cagr': 0.21069696112866643, 'mdd': 0.5282014485046166, 'win_rate': 0.3770491803278688, 'buy_time': 61, 'win_time': 23, 'gain_loss_ratio': 3.24, 'holding_time_ratio': 0.3010794140323824, 'investing_period': 2594, 'sharpe_ratio': 0.7545534388098796}
